PPO solving 2D Walker

In [ ]:
!pip install "gymnasium[box2d]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.9 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import imageio
import os
from google.colab import files

In [ ]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
# Actor Critic Network (shared backbone architecture)

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden=256):
        super().__init__()

        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
        )

        self.mu_head = nn.Linear(hidden, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
        self.value_head = nn.Linear(hidden, 1)

    def forward(self, x):
        z = self.shared(x)
        mu = self.mu_head(z)
        log_std = torch.clamp(self.log_std, -20, 2)
        std = torch.exp(log_std)
        value = self.value_head(z).squeeze(-1)
        return mu, std, value

    def get_action(self, state):
        mu, std, value = self.forward(state)
        dist = torch.distributions.Normal(mu, std)

        u = dist.rsample()
        action = torch.tanh(u)

        logp = dist.log_prob(u)
        logp -= torch.log(1 - action.pow(2) + 1e-6)
        logp = logp.sum(-1)

        entropy = dist.entropy().sum(-1)
        return action, logp, entropy, value

    def evaluate_actions(self, states, actions):
        mu, std, value = self.forward(states)
        dist = torch.distributions.Normal(mu, std)

        eps = 1e-6
        actions = torch.clamp(actions, -1 + eps, 1 - eps)
        u = 0.5 * torch.log((1 + actions) / (1 - actions))

        logp = dist.log_prob(u)
        logp -= torch.log(1 - actions.pow(2) + 1e-6)
        logp = logp.sum(-1)

        entropy = dist.entropy().sum(-1)
        return logp, entropy, value

    def act_deterministic(self, state):
        with torch.no_grad():
            mu, _, _ = self.forward(state)
            return torch.tanh(mu)


# Computing GAE

@torch.no_grad()
def compute_gae(rewards, dones, values, next_value, gamma=0.99, gae_lambda=0.95):
    T = rewards.shape[0]
    advantages = torch.zeros(T, device=rewards.device)

    gae = 0
    for t in reversed(range(T)):
        mask = 1.0 - dones[t]
        v_next = next_value if t == T - 1 else values[t + 1]
        delta = rewards[t] + gamma * v_next * mask - values[t]
        gae = delta + gamma * gae_lambda * mask * gae
        advantages[t] = gae

    returns = advantages + values
    return advantages.detach(), returns.detach()


# Training setup

seed = 0
set_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

env = gym.make("BipedalWalker-v3")
eval_env = gym.make("BipedalWalker-v3", render_mode="rgb_array")

os.makedirs("videos", exist_ok=True)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

initial_lr = 2.5e-4
model = ActorCritic(state_dim, action_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=initial_lr)

# PPO hyperparameters
gamma = 0.99
gae_lambda = 0.95
clip_eps = 0.2
vf_coef = 0.5
ent_coef = 0.0
max_grad_norm = 0.5
ppo_epochs = 10
mini_batch_size = 64

rollout_len = 2048
total_timesteps = 1_000_000
eval_interval = 20_000

global_step = 0
state, _ = env.reset()

# PPO training loop

while global_step < total_timesteps:

    # LR Annealing
    frac = 1.0 - (global_step / total_timesteps)
    lr_now = initial_lr * frac
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr_now

    states, actions, rewards, dones, values, logps = [], [], [], [], [], []
    rollout_rewards = []

    # Rollouts
    for _ in range(rollout_len):

        state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        action, logp, entropy, value = model.get_action(state_tensor)

        next_state, reward, terminated, truncated, _ = env.step(
            action.detach().cpu().numpy()
        )

        done = terminated or truncated

        states.append(state_tensor)
        actions.append(action.detach())
        rewards.append(torch.tensor(reward, dtype=torch.float32, device=device))
        dones.append(torch.tensor(float(done), dtype=torch.float32, device=device))
        values.append(value.detach())
        logps.append(logp.detach())

        rollout_rewards.append(reward)

        state = next_state
        global_step += 1

        if done:
            state, _ = env.reset()

        if global_step >= total_timesteps:
            break

    mean_rollout_reward = np.mean(rollout_rewards)

    states = torch.stack(states)
    actions = torch.stack(actions)
    rewards = torch.stack(rewards)
    dones = torch.stack(dones)
    values = torch.stack(values)
    logps_old = torch.stack(logps)

    with torch.no_grad():
        last_state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        _, _, next_value = model.forward(last_state_tensor)

    advantages, returns = compute_gae(
        rewards, dones, values, next_value, gamma, gae_lambda
    )

    # Advantage Norm
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)


    dataset_size = states.size(0)

    # Logging metrics
    approx_kl_total = 0
    clip_frac_total = 0
    explained_var_total = 0
    update_steps = 0

    # PPO main training loop
    for _ in range(ppo_epochs):

        indices = torch.randperm(dataset_size, device=device)

        for start in range(0, dataset_size, mini_batch_size):
            end = start + mini_batch_size
            batch_idx = indices[start:end]

            mb_states = states[batch_idx]
            mb_actions = actions[batch_idx]
            mb_advantages = advantages[batch_idx]
            mb_returns = returns[batch_idx]
            mb_logps_old = logps_old[batch_idx]
            mb_values_old = values[batch_idx]

            logps_new, entropy, values_new = model.evaluate_actions(
                mb_states, mb_actions
            )

            ratio = torch.exp(logps_new - mb_logps_old)

            surr1 = ratio * mb_advantages
            surr2 = torch.clamp(ratio, 1 - clip_eps, 1 + clip_eps) * mb_advantages    # PPO clipping

            policy_loss = -torch.min(surr1, surr2).mean()

            # Value function clipping
            value_clipped = mb_values_old + torch.clamp(
                values_new - mb_values_old,
                -clip_eps,
                clip_eps,
            )

            v_loss1 = (values_new - mb_returns).pow(2)
            v_loss2 = (value_clipped - mb_returns).pow(2)
            value_loss = 0.5 * torch.max(v_loss1, v_loss2).mean()

            entropy_loss = entropy.mean()

            # PPO Loss
            loss = policy_loss + vf_coef * value_loss - ent_coef * entropy_loss

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)      # Gradient clipping
            optimizer.step()

            # Logging stats
            approx_kl = (mb_logps_old - logps_new).mean().item()
            clip_frac = ((ratio - 1.0).abs() > clip_eps).float().mean().item()
            explained_var = 1 - torch.var(mb_returns - values_new) / (
                torch.var(mb_returns) + 1e-8
            )

            approx_kl_total += approx_kl
            clip_frac_total += clip_frac
            explained_var_total += explained_var.item()
            update_steps += 1

    print(
        f"Step: {global_step} | "
        f"LR: {lr_now:.6f} | "
        f"Reward: {mean_rollout_reward:.2f} | "
        f"KL: {approx_kl_total/update_steps:.5f} | "
        f"ClipFrac: {clip_frac_total/update_steps:.3f} | "
        f"ExplVar: {explained_var_total/update_steps:.3f}"
    )

    # Evaluation
    if global_step % eval_interval < rollout_len:

        s, _ = eval_env.reset()
        done = False
        ep_return = 0
        frames = []

        while not done:
            s_tensor = torch.tensor(s, dtype=torch.float32, device=device)

            # Determininstic evaluation
            action = model.act_deterministic(s_tensor)

            s, r, terminated, truncated, _ = eval_env.step(
                action.cpu().numpy()
            )
            done = terminated or truncated
            ep_return += r

            frame = eval_env.render()
            if frame is not None:
                frames.append(frame)

        print(f"[Eval @ {global_step}] Return: {ep_return:.2f}")

        video_path = f"videos/ppo_bipedal_eval_{global_step}.mp4"
        imageio.mimsave(video_path, frames, fps=30)
        print(f"Saved video: {video_path}")

env.close()
eval_env.close()


Device: cuda
Step: 2048 | LR: 0.000250 | Reward: -0.23 | KL: 0.02304 | ClipFrac: 0.235 | ExplVar: -0.000
Step: 4096 | LR: 0.000249 | Reward: -0.05 | KL: 0.01894 | ClipFrac: 0.209 | ExplVar: 0.170
Step: 6144 | LR: 0.000249 | Reward: -0.06 | KL: 0.02325 | ClipFrac: 0.240 | ExplVar: 0.210
Step: 8192 | LR: 0.000248 | Reward: -0.05 | KL: 0.01584 | ClipFrac: 0.208 | ExplVar: 0.319
Step: 10240 | LR: 0.000248 | Reward: -0.10 | KL: 0.01594 | ClipFrac: 0.172 | ExplVar: 0.014
Step: 12288 | LR: 0.000247 | Reward: -0.44 | KL: 0.01262 | ClipFrac: 0.177 | ExplVar: 0.015
Step: 14336 | LR: 0.000247 | Reward: -0.09 | KL: 0.01221 | ClipFrac: 0.156 | ExplVar: 0.130
Step: 16384 | LR: 0.000246 | Reward: -0.54 | KL: 0.01381 | ClipFrac: 0.160 | ExplVar: 0.034
Step: 18432 | LR: 0.000246 | Reward: -0.56 | KL: 0.00609 | ClipFrac: 0.171 | ExplVar: 0.019
Step: 20480 | LR: 0.000245 | Reward: -0.20 | KL: 0.01577 | ClipFrac: 0.182 | ExplVar: 0.051
[Eval @ 20480] Return: -95.56


Saved video: videos/ppo_bipedal_eval_20480.mp4
Step: 22528 | LR: 0.000245 | Reward: -0.05 | KL: 0.01056 | ClipFrac: 0.202 | ExplVar: 0.430
Step: 24576 | LR: 0.000244 | Reward: -0.09 | KL: 0.00944 | ClipFrac: 0.152 | ExplVar: 0.227
Step: 26624 | LR: 0.000244 | Reward: -0.18 | KL: 0.01009 | ClipFrac: 0.142 | ExplVar: 0.121
Step: 28672 | LR: 0.000243 | Reward: -0.04 | KL: 0.00915 | ClipFrac: 0.161 | ExplVar: 0.741
Step: 30720 | LR: 0.000243 | Reward: -0.05 | KL: 0.01667 | ClipFrac: 0.211 | ExplVar: 0.792
Step: 32768 | LR: 0.000242 | Reward: -0.15 | KL: 0.00566 | ClipFrac: 0.155 | ExplVar: 0.189
Step: 34816 | LR: 0.000242 | Reward: -0.14 | KL: 0.00936 | ClipFrac: 0.119 | ExplVar: 0.145
Step: 36864 | LR: 0.000241 | Reward: -0.05 | KL: 0.01305 | ClipFrac: 0.199 | ExplVar: 0.643
Step: 38912 | LR: 0.000241 | Reward: -0.04 | KL: 0.01315 | ClipFrac: 0.190 | ExplVar: 0.567
Step: 40960 | LR: 0.000240 | Reward: -0.09 | KL: 0.00915 | ClipFrac: 0.120 | ExplVar: 0.314


[Eval @ 40960] Return: -47.32
Saved video: videos/ppo_bipedal_eval_40960.mp4
Step: 43008 | LR: 0.000240 | Reward: -0.04 | KL: 0.01217 | ClipFrac: 0.183 | ExplVar: 0.592
Step: 45056 | LR: 0.000239 | Reward: -0.04 | KL: 0.00998 | ClipFrac: 0.145 | ExplVar: 0.861
Step: 47104 | LR: 0.000239 | Reward: -0.13 | KL: 0.00950 | ClipFrac: 0.105 | ExplVar: 0.285
Step: 49152 | LR: 0.000238 | Reward: -0.04 | KL: 0.01856 | ClipFrac: 0.204 | ExplVar: 0.842
Step: 51200 | LR: 0.000238 | Reward: -0.04 | KL: 0.01312 | ClipFrac: 0.175 | ExplVar: 0.874
Step: 53248 | LR: 0.000237 | Reward: -0.06 | KL: 0.01027 | ClipFrac: 0.137 | ExplVar: 0.822
Step: 55296 | LR: 0.000237 | Reward: -0.08 | KL: 0.00462 | ClipFrac: 0.145 | ExplVar: 0.415
Step: 57344 | LR: 0.000236 | Reward: -0.08 | KL: 0.01078 | ClipFrac: 0.141 | ExplVar: -0.000
Step: 59392 | LR: 0.000236 | Reward: -0.04 | KL: 0.01048 | ClipFrac: 0.170 | ExplVar: 0.760
Step: 61440 | LR: 0.000235 | Reward: -0.03 | KL: 0.01300 | ClipFrac: 0.178 | ExplVar: 0.903


[Eval @ 61440] Return: -27.27
Saved video: videos/ppo_bipedal_eval_61440.mp4
Step: 63488 | LR: 0.000235 | Reward: -0.09 | KL: 0.00149 | ClipFrac: 0.102 | ExplVar: 0.461
Step: 65536 | LR: 0.000234 | Reward: -0.03 | KL: 0.00608 | ClipFrac: 0.143 | ExplVar: 0.773
Step: 67584 | LR: 0.000234 | Reward: -0.02 | KL: 0.01257 | ClipFrac: 0.180 | ExplVar: 0.857
Step: 69632 | LR: 0.000233 | Reward: -0.02 | KL: 0.01288 | ClipFrac: 0.181 | ExplVar: 0.780
Step: 71680 | LR: 0.000233 | Reward: -0.01 | KL: 0.00810 | ClipFrac: 0.186 | ExplVar: 0.664
Step: 73728 | LR: 0.000232 | Reward: -0.01 | KL: 0.01480 | ClipFrac: 0.168 | ExplVar: 0.833
Step: 75776 | LR: 0.000232 | Reward: -0.01 | KL: 0.01177 | ClipFrac: 0.189 | ExplVar: 0.665
Step: 77824 | LR: 0.000231 | Reward: -0.01 | KL: 0.01634 | ClipFrac: 0.148 | ExplVar: 0.776
Step: 79872 | LR: 0.000231 | Reward: -0.00 | KL: 0.01815 | ClipFrac: 0.192 | ExplVar: 0.709
Step: 81920 | LR: 0.000230 | Reward: 0.01 | KL: 0.01244 | ClipFrac: 0.192 | ExplVar: 0.789


[Eval @ 81920] Return: 80.12
Saved video: videos/ppo_bipedal_eval_81920.mp4
Step: 83968 | LR: 0.000230 | Reward: -0.04 | KL: 0.01631 | ClipFrac: 0.150 | ExplVar: 0.396
Step: 86016 | LR: 0.000229 | Reward: 0.01 | KL: 0.01126 | ClipFrac: 0.186 | ExplVar: 0.810
Step: 88064 | LR: 0.000228 | Reward: 0.02 | KL: 0.01201 | ClipFrac: 0.189 | ExplVar: 0.783
Step: 90112 | LR: 0.000228 | Reward: 0.03 | KL: 0.01190 | ClipFrac: 0.164 | ExplVar: 0.826
Step: 92160 | LR: 0.000227 | Reward: 0.03 | KL: 0.01999 | ClipFrac: 0.215 | ExplVar: 0.823
Step: 94208 | LR: 0.000227 | Reward: 0.04 | KL: 0.01466 | ClipFrac: 0.199 | ExplVar: 0.880
Step: 96256 | LR: 0.000226 | Reward: 0.04 | KL: 0.01056 | ClipFrac: 0.181 | ExplVar: 0.797
Step: 98304 | LR: 0.000226 | Reward: -0.06 | KL: 0.01331 | ClipFrac: 0.175 | ExplVar: 0.320
Step: 100352 | LR: 0.000225 | Reward: 0.04 | KL: 0.01444 | ClipFrac: 0.185 | ExplVar: 0.862


...

[Eval @ 921600] Return: -55.34
Saved video: videos/ppo_bipedal_eval_921600.mp4
Step: 923648 | LR: 0.000020 | Reward: 0.25 | KL: 0.00472 | ClipFrac: 0.064 | ExplVar: 0.562
Step: 925696 | LR: 0.000019 | Reward: 0.27 | KL: 0.00660 | ClipFrac: 0.097 | ExplVar: 0.652
Step: 927744 | LR: 0.000019 | Reward: 0.21 | KL: 0.00290 | ClipFrac: 0.038 | ExplVar: 0.858
Step: 929792 | LR: 0.000018 | Reward: 0.21 | KL: 0.00493 | ClipFrac: 0.036 | ExplVar: 0.696
Step: 931840 | LR: 0.000018 | Reward: 0.26 | KL: 0.00584 | ClipFrac: 0.050 | ExplVar: 0.666
Step: 933888 | LR: 0.000017 | Reward: 0.28 | KL: 0.00578 | ClipFrac: 0.075 | ExplVar: 0.579
Step: 935936 | LR: 0.000017 | Reward: 0.20 | KL: 0.00430 | ClipFrac: 0.072 | ExplVar: 0.721
Step: 937984 | LR: 0.000016 | Reward: 0.28 | KL: 0.00917 | ClipFrac: 0.040 | ExplVar: 0.495
Step: 940032 | LR: 0.000016 | Reward: 0.26 | KL: 0.00617 | ClipFrac: 0.085 | ExplVar: 0.585


[Eval @ 940032] Return: 66.89
Saved video: videos/ppo_bipedal_eval_940032.mp4
Step: 942080 | LR: 0.000015 | Reward: 0.22 | KL: -0.00026 | ClipFrac: 0.038 | ExplVar: 0.736
Step: 944128 | LR: 0.000014 | Reward: 0.27 | KL: 0.00402 | ClipFrac: 0.097 | ExplVar: 0.605
Step: 946176 | LR: 0.000014 | Reward: 0.26 | KL: 0.01220 | ClipFrac: 0.104 | ExplVar: 0.557
Step: 948224 | LR: 0.000013 | Reward: 0.21 | KL: 0.00369 | ClipFrac: 0.047 | ExplVar: 0.816
Step: 950272 | LR: 0.000013 | Reward: 0.27 | KL: 0.00101 | ClipFrac: 0.036 | ExplVar: 0.630
Step: 952320 | LR: 0.000012 | Reward: 0.26 | KL: 0.00405 | ClipFrac: 0.036 | ExplVar: 0.597
Step: 954368 | LR: 0.000012 | Reward: 0.29 | KL: 0.00346 | ClipFrac: 0.042 | ExplVar: 0.634
Step: 956416 | LR: 0.000011 | Reward: 0.19 | KL: 0.00250 | ClipFrac: 0.018 | ExplVar: 0.736
Step: 958464 | LR: 0.000011 | Reward: 0.27 | KL: 0.00170 | ClipFrac: 0.022 | ExplVar: 0.606
Step: 960512 | LR: 0.000010 | Reward: 0.28 | KL: 0.00553 | ClipFrac: 0.093 | ExplVar: 0.641


[Eval @ 960512] Return: 24.49
Saved video: videos/ppo_bipedal_eval_960512.mp4
Step: 962560 | LR: 0.000010 | Reward: 0.16 | KL: 0.00161 | ClipFrac: 0.028 | ExplVar: 0.855
Step: 964608 | LR: 0.000009 | Reward: 0.27 | KL: 0.00635 | ClipFrac: 0.021 | ExplVar: 0.568
Step: 966656 | LR: 0.000009 | Reward: 0.28 | KL: 0.00521 | ClipFrac: 0.055 | ExplVar: 0.605
Step: 968704 | LR: 0.000008 | Reward: 0.27 | KL: 0.00106 | ClipFrac: 0.048 | ExplVar: 0.579
Step: 970752 | LR: 0.000008 | Reward: 0.22 | KL: 0.00270 | ClipFrac: 0.023 | ExplVar: 0.718
Step: 972800 | LR: 0.000007 | Reward: 0.12 | KL: 0.00166 | ClipFrac: 0.012 | ExplVar: 0.845
Step: 974848 | LR: 0.000007 | Reward: 0.26 | KL: 0.00258 | ClipFrac: 0.049 | ExplVar: 0.483
Step: 976896 | LR: 0.000006 | Reward: 0.10 | KL: 0.00710 | ClipFrac: 0.030 | ExplVar: 0.868
Step: 978944 | LR: 0.000006 | Reward: 0.28 | KL: 0.00242 | ClipFrac: 0.013 | ExplVar: 0.500
Step: 980992 | LR: 0.000005 | Reward: 0.14 | KL: 0.00117 | ClipFrac: 0.001 | ExplVar: 0.858


[Eval @ 980992] Return: 83.95
Saved video: videos/ppo_bipedal_eval_980992.mp4
Step: 983040 | LR: 0.000005 | Reward: 0.21 | KL: 0.00443 | ClipFrac: 0.010 | ExplVar: 0.412
Step: 985088 | LR: 0.000004 | Reward: 0.15 | KL: 0.00181 | ClipFrac: 0.002 | ExplVar: 0.873
Step: 987136 | LR: 0.000004 | Reward: 0.21 | KL: 0.00129 | ClipFrac: 0.015 | ExplVar: 0.767
Step: 989184 | LR: 0.000003 | Reward: 0.21 | KL: -0.00006 | ClipFrac: 0.011 | ExplVar: 0.761
Step: 991232 | LR: 0.000003 | Reward: 0.27 | KL: 0.00080 | ClipFrac: 0.000 | ExplVar: 0.641
Step: 993280 | LR: 0.000002 | Reward: 0.20 | KL: 0.00128 | ClipFrac: 0.001 | ExplVar: 0.723
Step: 995328 | LR: 0.000002 | Reward: 0.27 | KL: -0.00012 | ClipFrac: 0.000 | ExplVar: 0.604
Step: 997376 | LR: 0.000001 | Reward: 0.27 | KL: -0.00014 | ClipFrac: 0.000 | ExplVar: 0.512
Step: 999424 | LR: 0.000001 | Reward: 0.15 | KL: -0.00008 | ClipFrac: 0.000 | ExplVar: 0.854
Step: 1000000 | LR: 0.000000 | Reward: 0.27 | KL: -0.00001 | ClipFrac: 0.000 | ExplVar: 0.

[Eval @ 1000000] Return: 4.07
Saved video: videos/ppo_bipedal_eval_1000000.mp4


In [ ]:
!zip -r videos.zip /content/videos

  adding: content/videos/ (stored 0%)
  adding: content/videos/ppo_bipedal_eval_120832.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_921600.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_520192.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_901120.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_821248.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_481280.mp4 (deflated 4%)
  adding: content/videos/ppo_bipedal_eval_241664.mp4 (deflated 4%)
  adding: content/videos/ppo_bipedal_eval_40960.mp4 (deflated 38%)
  adding: content/videos/ppo_bipedal_eval_161792.mp4 (deflated 5%)
  adding: content/videos/ppo_bipedal_eval_221184.mp4 (deflated 5%)
  adding: content/videos/ppo_bipedal_eval_1000000.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_860160.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_841728.mp4 (deflated 3%)
  adding: content/videos/ppo_bipedal_eval_380928.mp4 (deflated 3%)
  adding: content/video

In [ ]:
files.download('/content/videos.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>